In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("key_value_rdd_op_joins")
sc = SparkContext(conf = conf)

## Operations

### groupByKey
* `k_v_RDD.groupByKey(numPartitions=None, partitionFunc=<function portable_hash>)`
* 주어지는 key를 기준으로 Group을 만들어 줍니다.
* Transformations 함수 입니다.

In [2]:
rdd = sc.parallelize([
    ("짜장면", 15),
    ("짬뽕", 10),
    ("짜장면", 5)
])

In [3]:
rdd.groupByKey().mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 1)]

In [4]:
rdd.groupByKey().mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10])]

`groupBy` VS `groupByKey`

In [5]:
# groupBy는 그룹핑할 키에 대한 정의를 개발자가 직접 해줘야 한다.
grouped = sc.parallelize([
    "C", "C++", "Python", "Java", "C#"
]).groupBy(lambda x: x[0]).mapValues(list)

grouped.collect()

[('C', ['C', 'C++', 'C#']), ('P', ['Python']), ('J', ['Java'])]

In [6]:
# groupByKey는 K-V RDD를 사용할 때 Key가 알아서 그룹핑의 기준이 된다.
x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)
], 3)

y = x.groupByKey()

In [7]:
# 파티션 개수 확인하기
y.getNumPartitions()

3

In [8]:
y.mapValues(list).collect()

[('MATH', [7, 2, 8, 3]), ('ENGLISH', [7, 4, 9, 4]), ('SCIENCE', [7, 6, 9, 5])]

# reduceByKey
- `KeyValueRDD.reduceByKey(<task>, numPartitions=None, partitionFunc=<function portable_hash>)`
- 주어지는 `key`를 기준으로 `Group`을 만들고 합쳐(`task`대로)줍니다.
- Transformations 함수 입니다.

In [9]:
from operator import add

rdd = sc.parallelize([
    ("짜장면", 15),
    ("짬뽕", 10),
    ("짜장면", 5)
])

rdd.reduceByKey(add).collect()

[('짜장면', 20), ('짬뽕', 10)]

개념적으로는 `groupByKey` + `reduce` 입니다.

# mapValues
- `KeyValueRDD.mapValues(<task>)`
- 함수를 `Value`에만 적용합니다.
    - 파티션과 키는 그 위치에 그대로 있습니다.
- `Transformations`

In [10]:
rdd = sc.parallelize([
    ("하의", ["청바지", "반바지", "치마"]),
    ("상의", ["니트", "반팔", "긴팔", "나시"])
])

rdd.mapValues(len).collect()

[('하의', 3), ('상의', 4)]

# countByKey
- `KeyValueRDD.countByKey(<task>)`
- 각 키가 가진 요소들의 개수를 센다.
- `Action`

In [13]:
rdd.countByKey()

defaultdict(int, {'하의': 1, '상의': 1})

# keys()
- 모든 key를 가진 RDD를 생성합니다.
- `keys()`는 파티션을 유지하거나 키가 굉장히 많은 경우가 있기 때문에 Transformations 입니다.

In [14]:
rdd.keys()

PythonRDD[35] at RDD at PythonRDD.scala:53

In [15]:
rdd.keys().collect()

['하의', '상의']

In [17]:
x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)
], 3)

print(x.keys().count())
print(x.keys().distinct().count())

12
3


# Joins
- `Inner Join` : 서로간에 존재하는 키만 합쳐준다.
- `Outer Join` : 기준이 되는 한 쪽에는 데이터가 있고, 다른 쪽에는 데이터가 없는 경우
    - 설정한 기준에 따라서 기준에 맞는 데이터가 항상 남아있는다.
    - `leftOuterJoin` : 왼쪽에 있는 rdd가 기준이 된다.( 함수를 호출 하는 쪽 )
    - `rightOuterJoin` : 오른쪽에 있는 rdd가 기준이 된다. ( 함수에 매개변수로 들어가는 쪽 )

In [18]:
rdd1 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("hoo", 3)
])

rdd2 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("goo", 10),
    ("moo", 6)
])

In [19]:
# Inner Join
rdd1.join(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10))]

In [20]:
# Left Outer Join
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10)), ('hoo', (3, None))]

In [21]:
# Right Outer Join
rdd1.rightOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('moo', (None, 6)), ('goo', (2, 2)), ('goo', (2, 10))]

In [22]:
sc.stop()